<a href="https://colab.research.google.com/github/june12mayank/Machine_learning_notebooks/blob/master/skillenza%20classification%20comptetion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
from scipy import stats
from scipy.stats import norm, skew
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [0]:
sample=pd.read_csv('/content/SAMPLE.csv')
test=pd.read_csv('/content/TEST.csv')
train=pd.read_csv('/content/TRAINING.csv')

In [56]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 14 columns):
id             7000 non-null int64
Area(total)    7000 non-null int64
Troom          6998 non-null float64
Nbedrooms      6997 non-null float64
Nbwashrooms    6998 non-null float64
Twashrooms     6999 non-null float64
roof           5647 non-null object
Roof(Area)     5647 non-null float64
Lawn(Area)     6998 non-null float64
Nfloors        7000 non-null int64
API            6999 non-null float64
ANB            7000 non-null int64
Grade          7000 non-null object
Price          7000 non-null object
dtypes: float64(7), int64(4), object(3)
memory usage: 765.8+ KB


In [31]:
train_id=train['id']
test_id=test['id']
print(train.shape,test.shape)

(7000, 14) (3299, 13)


In [33]:
ntrain=train.shape[0]
ntest=test.shape[0]
y_train=train.Grade.values
df=pd.concat((train,test)).reset_index(drop=True)
df.drop(['Grade'],axis=1,inplace=True)
df.shape

(10299, 13)

In [35]:
all_data_na = (df.isnull().sum() / len(df)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio
roof,19.477619
Roof(Area),19.477619
Nbedrooms,0.038839
Troom,0.029129
Nbwashrooms,0.029129
Lawn(Area),0.029129
API,0.019419
Twashrooms,0.009710


In [0]:
df['roof']=df['roof'].fillna('NO')

In [0]:
df['Roof(Area)']=df['Roof(Area)'].fillna(0.0)

In [0]:
#because of correlation in Tbathrooms and bedrooms but then boxplot
df["Nbedrooms"] = df['Nbedrooms'].fillna(df['Nbedrooms'].mean())

In [0]:
df['Troom']=df['Troom'].fillna(df['Troom'].mean())

In [0]:
df['Nbwashrooms']=df['Nbwashrooms'].fillna(df['Nbwashrooms'].mean())

In [0]:
#no corr and also area!=3*lawn which i presume intially
df['Lawn(Area)']=df['Lawn(Area)'].fillna(df['Lawn(Area)'].mean())

In [0]:
df['API']=df['API'].fillna(df['API'].mean())

In [0]:
df['Twashrooms']=df['Twashrooms'].fillna(df['Twashrooms'].mean())

In [0]:
float_cols=['API','Lawn(Area)','Nbedrooms','Nbwashrooms','Roof(Area)','Troom','Twashrooms']
df[float_cols]=df[float_cols].applymap(np.int64)


In [0]:
df['roof']= df['roof'].map({'NO':0,'no':0,'YES':1,'yes':1})

In [0]:
df.Price=df.Price.str.rstrip('$').apply(np.int64)

In [0]:
from sklearn.preprocessing import LabelEncoder

lbl=LabelEncoder()
lbl.fit(list(y_train))
y_train = lbl.transform(list(y_train))

In [0]:
train = df[:ntrain]
test = df[ntrain:]

In [0]:
test_id=test['id']
test=test.drop('id',axis=1)

In [0]:
train=train.drop('id',axis=1)

In [164]:
model1=xgb.XGBClassifier()
model1.fit(train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
pred=model1.predict(test)

In [166]:
pred

array([1, 3, 2, ..., 2, 3, 3])

In [0]:
pred=lbl.inverse_transform(pred)

In [168]:
pred

array(['B', 'D', 'C', ..., 'C', 'D', 'D'], dtype='<U1')

In [171]:
pred.shape

(3299,)

In [174]:
sub=pd.DataFrame()
sub['id']=test_id
sub['Grade']=pred
sub.head()

,id,Grade
7000,1,B
7001,2,D
7002,3,C
7003,4,C
7004,5,D


In [0]:
sub.to_csv('submission_jai_ho.csv',index=False)

In [175]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3299 entries, 7000 to 10298
Data columns (total 2 columns):
id       3299 non-null int64
Grade    3299 non-null object
dtypes: int64(1), object(1)
memory usage: 51.7+ KB


In [173]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3299 entries, 0 to 3298
Data columns (total 2 columns):
id       3299 non-null int64
Grade    3299 non-null object
dtypes: int64(1), object(1)
memory usage: 51.7+ KB


In [158]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 12 columns):
ANB            7000 non-null int64
API            7000 non-null int64
Area(total)    7000 non-null int64
Lawn(Area)     7000 non-null int64
Nbedrooms      7000 non-null int64
Nbwashrooms    7000 non-null int64
Nfloors        7000 non-null int64
Price          7000 non-null int64
Roof(Area)     7000 non-null int64
Troom          7000 non-null int64
Twashrooms     7000 non-null int64
roof           7000 non-null int64
dtypes: int64(12)
memory usage: 656.4 KB
